Algoritmos Genéticos que resolvem Sudoku 4x4
========================================



In [1]:
#importação de bibliotecas
from random import randint, choice, uniform
import copy

In [2]:
# Função para criação do tabuleiro seguindo o template pré-definido acima
def tabuleiro()-> list:
    
    matriz = [
        [0, 0, 0, 4],
        [0, 0, 0, 0],
        [2, 0, 0, 3],
        [4, 0, 1, 2]
    ]
    return matriz

#Nível Fácil
    #matriz = [
    #        [0, 0, 0, 4],
    #        [0, 0, 0, 0],
    #        [2, 0, 0, 3],
    #        [4, 0, 1, 2]
    #    ]
    #    return matriz

#Nível Médio
    #matriz = [
    #        [0, 0, 0, 3],
    #        [0, 4, 0, 0],
    #        [1, 0, 0, 4],
    #        [0, 0, 3, 0]
    #    ]
    #    return matriz
    
#Nível Difícil
    #matriz = [
    #        [0, 0, 0, 3],
    #        [0, 4, 0, 0],
    #        [0, 0, 3, 2],
    #        [0, 0, 0, 0]
    #    ]
    #    return matriz

In [3]:
# Função para completar os espaços em branco com números aleatórios
def preencher(quadro: list, espacos: list) -> list:
    def validacao(quadro: list, num: int, row: int, col: int) -> bool:
        # Verifica se o número já está presente na mesma linha
        for i in range(4):
            if quadro[row][i] == num:
                return False
        
        # Verifica se o número já está presente na mesma coluna
        for i in range(4):
            if quadro[i][col] == num:
                return False
        
        # Verifica se o número já está presente no mesmo bloco 2x2
        start_row = (row // 2) * 2
        start_col = (col // 2) * 2
        for i in range(start_row, start_row + 2):
            for j in range(start_col, start_col + 2):
                if quadro[i][j] == num:
                    return False
        
        return True

    def solve(quadro: list, espacos: list) -> bool:
        if len(espacos) == 0:
            return True
        
        row, col = espacos[0]
        for num in range(1, 5):
            if validacao(quadro, num, row, col):
                quadro[row][col] = num
                if solve(quadro, espacos[1:]):
                    return True
                quadro[row][col] = 0
        
        return False

    solve(quadro, espacos)
    return quadro

In [4]:
# Função para exibir o tabuleiro de forma clara para o usuário
def exibe(quadro: list):
    print(f"|{quadro[0][0]} {quadro[0][1]}| |{quadro[1][0]} {quadro[1][1]}|")
    print(f"|{quadro[0][2]} {quadro[0][3]}| |{quadro[1][2]} {quadro[1][3]}|", end='\n\n')
    
    print(f"|{quadro[2][0]} {quadro[2][1]}| |{quadro[3][0]} {quadro[3][1]}|")
    print(f"|{quadro[2][2]} {quadro[2][3]}| |{quadro[3][2]} {quadro[3][3]}|", end='\n\n')

In [5]:
# Funções para localizar os espaços em branco
def encontra_espacos(estado_atual: list) -> list:
    espacos = []
    for submatriz in range(len(estado_atual)):
        for celula in range (len(estado_atual[submatriz])):
            if estado_atual[submatriz][celula] == 0:
                espacos.append((submatriz, celula))
    return espacos

In [6]:
# Função para calcular a quantidade de conflitos no tabuleiro
def calcular_conflitos(estado_atual: list) -> int:
    conflitos = 0
    
    # Verificando conflitos dentro das submatrizes
    for submatriz in range (len(estado_atual)):
        numeros_conflitantes_submatriz = []
        for celula in range (len(estado_atual[submatriz])):
            numero = estado_atual[submatriz][celula]
            if numero not in numeros_conflitantes_submatriz:
                quantidade_conflitos_submatrizes = estado_atual[submatriz].count(numero)
                if quantidade_conflitos_submatrizes > 1:
                    conflitos += quantidade_conflitantes_submatrizes-1
                    numeros_conflitantes_submatriz.append(numero)
    
    # Verificando conflito na linha
    linhas = matriz_linhas(estado_atual)
    for linha in range(len(linhas)):
        numeros_conflitantes_linhas = []
        for celula in range (len(linhas[linha])):
            numero = linhas[linha][celula]
            if numero not in numeros_conflitantes_linhas:
                quantidade_conflitantes_linha = linhas[linha].count(numero)
                if quantidade_conflitantes_linha > 1:
                    conflitos += quantidade_conflitantes_linha-1
                    numeros_conflitantes_linhas.append(numero)
                    
    # Verificando conflitos na coluna
    colunas = matriz_colunas(estado_atual)
    for coluna in range(len(colunas)):
        numeros_conflitantes_colunas = []
        for celula in range(len(colunas[coluna])):
            numero = colunas[coluna][celula]
            if numero not in numeros_conflitantes_colunas:
                quantidade_conflitantes_coluna = colunas[coluna].count(numero)
                if quantidade_conflitantes_coluna > 1:
                    conflitos += quantidade_conflitantes_coluna-1
                    numeros_conflitantes_colunas.append(numero)
            
    return conflitos

In [7]:
# Função para criar as linhas do tabuleiro para verificação de conflito
def matriz_linhas(estado_atual: list) -> list:
    linhas = []
    
    linhas.append(estado_atual[0][0:2] + estado_atual[1][0:2])
    linhas.append(estado_atual[0][2:4] + estado_atual[1][2:4])
    linhas.append(estado_atual[2][0:2] + estado_atual[3][0:2])
    linhas.append(estado_atual[2][2:4] + estado_atual[3][2:4])
    
    return linhas

In [8]:
def matriz_colunas(estado_atual: list) -> list:
    colunas = []
    
    colunas.append([estado_atual[0][0], estado_atual[0][2], estado_atual[2][0], estado_atual[2][2]])
    colunas.append([estado_atual[0][1], estado_atual[0][3], estado_atual[2][1], estado_atual[2][3]])
    colunas.append([estado_atual[1][0], estado_atual[1][2], estado_atual[3][0], estado_atual[3][2]])
    colunas.append([estado_atual[1][1], estado_atual[1][3], estado_atual[3][1], estado_atual[3][3]])
    
    return colunas

In [9]:
# Função para gerar a população inicial
#def populacao_inicial(tamanho_populacao: int, lista_espacos_vazios: list) -> list:
    #populacao = []
    #for individuo in range(tamanho_populacao):
        #quadro = tabuleiro()
        #quadro = preencher(quadro, lista_espacos_vazios)
        #populacao.append(quadro)
    #return populacao

def populacao_inicial(tamanho_populacao: int, lista_espacos_vazios: list) -> list:
    populacao = []
    for individuo in range(tamanho_populacao):
        quadro = tabuleiro()
        quadro_preenchido = preencher(quadro, lista_espacos_vazios)
        populacao.append(quadro_preenchido)
    return populacao

In [10]:
# Função de Cross Over (Cruzamento) de indivíduos
def cruzamento(genoma_1: list, genoma_2: list) -> list:
    g_1 = copy.deepcopy(genoma_1)
    g_2 = copy.deepcopy(genoma_2)
    ponto_corde = randint(0, len(g_1)-1)
    return g_1[:ponto_corde] + g_2[ponto_corde:], g_2[:ponto_corde] + g_1[ponto_corde:]

In [11]:
# Função de Mutação que altera uma quantidade específica de elementos da submatriz
def mutacao(estado_atual: list, quantidade_mutacoes: int, lista_espacos_vazios: list) -> list:
    lista_espacos_vazios_local = copy.deepcopy(lista_espacos_vazios)
    for quantidade in range(quantidade_mutacoes):
        espaco = choice(lista_espacos_vazios_local)
        lista_espacos_vazios_local.remove(espaco)
        numero = randint(1, 4)
        while numero == estado_atual[espaco[0]][espaco[1]]:
            numero = randint(1, 4)
        estado_atual[espaco[0]][espaco[1]] = numero
    return estado_atual

In [12]:
# Função fitness para avaliar a população
def fitness(populacao: list) -> list:
    valores_fitness = []
    for individuo in populacao:
        valores_fitness.append(1 - calcular_conflitos(individuo)/36)
    return valores_fitness

In [13]:
# Função de Roleta Viciada para selecionar indivíduos
def roleta_viciada(populacao: list, fitness: list) -> list:
    fitness_total = sum(fitness)
    fracoes = [f/fitness_total for f in fitness]
    random_numero = uniform(0,1)
    acum = 0
    for i, individuo in enumerate(populacao):
        acum += fracoes[i]
        if acum >= random_numero:
            return individuo
        
    return choice(populacao)

In [14]:
# Função utilizada para selecionar uma determinada quantidade de casais reprodutores
def selecao_natural(populacao: list, casais_reprodutores: int) -> list:
    selection = []
    valor_fitness = fitness(populacao)
    for individuo in range(2*casais_reprodutores):
        selection.append(roleta_viciada(populacao, valor_fitness))
    return selection

In [15]:
# Função para verificar se a meta foi alcançada
def meta_teste(populacao: list):
    valor_fitness = fitness(populacao)
    try:
        posicao_meta = valor_fitness.index(1)
        return populacao[posicao_meta]
    except (ValueError):
        return -1

In [16]:
# Função para localizar o melhor indivíduo da geração
def melhor_individuo(populacao: list) -> list:
    valor_fitness = fitness(populacao)
    posicao = valor_fitness.index(max(valor_fitness))
    return (populacao[posicao], valor_fitness[posicao])

In [17]:
# Função do Algoritmo Genético
def genetico(tamanho_populacao: int, geracoes: int, lista_espacos_vazios: list, quantidade_mutacoes: int, porcentagem_mutacao: int, casais_reprodutores: int):
    
    populacao = populacao_inicial(tamanho_populacao, lista_espacos_vazios)
    best = melhor_individuo(populacao)
    print(f" Melhor indivíduo da geração 0 com fitness: {best[1]}")
    
    for geracao in range(geracoes):
        print(f"Geração: {geracao}")
        
        meta = meta_teste(populacao)
        if meta != -1:
            print(f"\nSolução Encontrada na Geração: {geracao}\n")
            return exibe(meta)
        else:
            best_geracao = melhor_individuo(populacao)
            if best_geracao[1] > best[1]:
                best = best_geration
                print(f'Novo melhor fitness com valor: {best[1]}')
        
        nova_populacao = []
        reprodutores = selecao_natural(populacao, casais_reprodutores)
        pais = reprodutores[:casais_reprodutores]
        maes = reprodutores[casais_reprodutores:]
        for pai, mae in zip(pais, maes):
            filho_1, filho_2 = cruzamento(pai, mae)
            nova_populacao.append(filho_1)
            nova_populacao.append(filho_2)
            
        meta = meta_teste(nova_populacao)
        if meta != -1:
            print(f"\nSolução Encontrada na Geração: {geracao}\n")
            return exibe(meta)
        else:
            best_geration = melhor_individuo(nova_populacao)
            if best_geration[1] > best[1]:
                best = best_geration
                print(f"Novo Melhor fitness com valor: {best[1]}")
        
        for individuo in nova_populacao:
            if randint(0, 100) < porcentagem_mutacao:
                individuo = mutacao(individuo, quantidade_mutacoes, lista_espacos_vazios)
        
        populacao = nova_populacao
        
    print(f"\nALGORITMO ENCERRADO DEVIDO AO NÚMERO DE GERAÇÕES SEM ENCONTRAR UM RESULTADO SATISFATÓRIO.\nO MELHOR INDIVÍDUO OBTEVE O FITNESS DE {best[1]}.\n\n")
    return exibe(best[0])

In [18]:
# Configurações iniciais do algoritmo genético
tamanho_populacao = 1000
geracoes = 10000
lista_espacos_vazios = encontra_espacos(tabuleiro())

quantidade_mutacoes = 5
if quantidade_mutacoes > len(lista_espacos_vazios):
    raise ValueError ("O número de valores alterado por mutação é maior que os espaços vazios!")
    
porcentagem_mutacao = 50
if porcentagem_mutacao > 100:
    raise ValueError ("A porcentagem de chance de mutação é maior que 100%")

casais_reprodutores = 4

# inicialização do algoritmo genético
genetico(tamanho_populacao, geracoes, lista_espacos_vazios, quantidade_mutacoes, porcentagem_mutacao, casais_reprodutores)

 Melhor indivíduo da geração 0 com fitness: 1.0
Geração: 0

Solução Encontrada na Geração: 0

|1 2| |3 4|
|3 4| |2 1|

|2 1| |4 3|
|4 3| |1 2|



### 